# Cargo los JSON , los desanido y guardo los dataframes desanidados en archivos parquet

# Archivo Reviews

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import files



## Importo el archivo JSON original reviews y luego lo desanido

In [4]:
import pandas as pd

In [5]:
import ast


ruta_reviews = '/content/drive/MyDrive/PI MLOps - STEAM_nuevo/Datasets/australian_user_reviews.json'

# Leer  el contenido del archivo reviews.json

user_reviews = []

with open(ruta_reviews, 'r', encoding = 'MacRoman') as f :
  for line in f.readlines():
    user_reviews.append(ast.literal_eval(line))

# Crear el DataFrame anidado :

df_reviews_anidado = pd.DataFrame(user_reviews)



In [6]:
df_reviews_anidado.head(1).reviews

0    [{'funny': '', 'posted': 'Posted November 5, 2...
Name: reviews, dtype: object

In [7]:
df_reviews_anidado.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [8]:
df_reviews_anidado[['reviews']].head(1)

,reviews
0,"[{'funny': '', 'posted': 'Posted November 5, 2..."


In [9]:
# Desanida la columna anidada "reviews"


reviews_desanidado = []

for index , row in df_reviews_anidado.iterrows():
    user_id = row['user_id']
    user_url = row['user_url']
    reviews = row['reviews']
    for i in reviews :
        new_row = {
            'user_id' : user_id,
            'user_url' : user_url,
            'funny' : i.get('funny', ''),
            'posted' : i.get('posted'),
            'last_edited' : i.get('last_edited', ''),
            'item_id' : i.get('item_id'),
            'helpful' : i.get('helpful'),
            # Elimina el valor predeterminado de la columna `recommended`
            'recommend' : bool(i.get('recommend')), # Son valores True o False
            'review' : i.get('review')

        }
        reviews_desanidado.append(new_row)

df_user_reviews = pd.DataFrame(reviews_desanidado)

# Convierte los valores de la columna `recommend` a tipo de datos booleano
df_user_reviews['recommend'] = df_user_reviews['recommend'].astype(bool)



In [10]:
df_user_reviews[['recommend']].value_counts()

recommend
True         52473
False         6832
dtype: int64

In [11]:
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59305 non-null  object
 1   user_url     59305 non-null  object
 2   funny        59305 non-null  object
 3   posted       59305 non-null  object
 4   last_edited  59305 non-null  object
 5   item_id      59305 non-null  object
 6   helpful      59305 non-null  object
 7   recommend    59305 non-null  bool  
 8   review       59305 non-null  object
dtypes: bool(1), object(8)
memory usage: 3.7+ MB


In [12]:
df_user_reviews.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [13]:
df_user_reviews.tail()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
59300,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
59301,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
59302,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59303,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,,Posted July 20.,,730,No ratings yet,True,:D
59304,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,,Posted July 2.,,440,No ratings yet,True,so much fun :D


## Guardo la version sin ETL , solo desanidado , ya que luego haré el EDA con esta version

In [14]:
df_user_reviews.to_parquet('reviews_raw.parquet')

## Un poco de limpieza de datos

## Elimino columnas que seran innecesarias para construir la API y el modelo

In [15]:
df_user_reviews.drop(['helpful', 'funny', 'user_url', 'last_edited', ], axis=1, inplace = True)

In [16]:
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    59305 non-null  object
 1   posted     59305 non-null  object
 2   item_id    59305 non-null  object
 3   recommend  59305 non-null  bool  
 4   review     59305 non-null  object
dtypes: bool(1), object(4)
memory usage: 1.9+ MB


In [17]:
df_user_reviews['user_id'] = df_user_reviews['user_id'].astype('str')

In [18]:
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    59305 non-null  object
 1   posted     59305 non-null  object
 2   item_id    59305 non-null  object
 3   recommend  59305 non-null  bool  
 4   review     59305 non-null  object
dtypes: bool(1), object(4)
memory usage: 1.9+ MB


In [19]:
df_user_reviews.head(2)

,user_id,posted,item_id,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.


## Convertimos la columna posted a un tipo valido para fecha

In [20]:
import pandas as pd

# Supongamos que ya tienes un DataFrame llamado df con tus datos

# Definir una función para convertir el formato actual en "%d %m %Y"
def convert_to_datetime(text):
    try:
        # Dividir el texto y eliminar "Posted" y el punto final
        date_str = text.split('Posted ')[1].rstrip('.')
        # Convertir la fecha en el nuevo formato
        date = pd.to_datetime(date_str, format='%B %d, %Y')
        # Formatear la fecha en "%d %m %Y"
        formatted_date = date.strftime('%d-%m-%Y')
        return formatted_date
    except:
        return None

# Aplicar la función a la columna "posted" y crear una nueva columna "formatted_posted"
df_user_reviews['posted'] = df_user_reviews['posted'].apply(convert_to_datetime)




In [21]:
df_user_reviews.head()

,user_id,posted,item_id,recommend,review
0,76561197970982479,05-11-2011,1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,15-07-2011,22200,True,It's unique and worth a playthrough.
2,76561197970982479,21-04-2011,43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,24-06-2014,251610,True,I know what you think when you see this title ...
4,js41637,08-09-2013,227300,True,For a simple (it's actually not all that simpl...


In [22]:
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    59305 non-null  object
 1   posted     49186 non-null  object
 2   item_id    59305 non-null  object
 3   recommend  59305 non-null  bool  
 4   review     59305 non-null  object
dtypes: bool(1), object(4)
memory usage: 1.9+ MB


## Eliminamos las reviews , las cuales no tiene un tipo valido de fecha , ya que la API , consultará la fecha

In [23]:
df_user_reviews.dropna(inplace = True)

In [24]:
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49186 entries, 0 to 59276
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    49186 non-null  object
 1   posted     49186 non-null  object
 2   item_id    49186 non-null  object
 3   recommend  49186 non-null  bool  
 4   review     49186 non-null  object
dtypes: bool(1), object(4)
memory usage: 1.9+ MB


In [25]:
df_reviews = df_user_reviews

In [26]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49186 entries, 0 to 59276
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    49186 non-null  object
 1   posted     49186 non-null  object
 2   item_id    49186 non-null  object
 3   recommend  49186 non-null  bool  
 4   review     49186 non-null  object
dtypes: bool(1), object(4)
memory usage: 1.9+ MB


In [27]:
df_reviews.head()

,user_id,posted,item_id,recommend,review
0,76561197970982479,05-11-2011,1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,15-07-2011,22200,True,It's unique and worth a playthrough.
2,76561197970982479,21-04-2011,43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,24-06-2014,251610,True,I know what you think when you see this title ...
4,js41637,08-09-2013,227300,True,For a simple (it's actually not all that simpl...


In [28]:
# Dividir la columna 'posted' por el guion '-'
df_reviews['year_posted'] = df_reviews['posted'].str.split('-').str[2]


In [29]:
df_reviews.head()

,user_id,posted,item_id,recommend,review,year_posted
0,76561197970982479,05-11-2011,1250,True,Simple yet with great replayability. In my opi...,2011
1,76561197970982479,15-07-2011,22200,True,It's unique and worth a playthrough.,2011
2,76561197970982479,21-04-2011,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011
3,js41637,24-06-2014,251610,True,I know what you think when you see this title ...,2014
4,js41637,08-09-2013,227300,True,For a simple (it's actually not all that simpl...,2013


In [30]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49186 entries, 0 to 59276
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      49186 non-null  object
 1   posted       49186 non-null  object
 2   item_id      49186 non-null  object
 3   recommend    49186 non-null  bool  
 4   review       49186 non-null  object
 5   year_posted  49186 non-null  object
dtypes: bool(1), object(5)
memory usage: 2.3+ MB


## Guardamos el archivo en formato parquet

In [31]:
df_reviews.to_parquet('reviews.parquet')

# Importamos el archivo steam y lo desanidamos

In [32]:
import pandas as pd


In [33]:
# Leer el  output_steam_games.json
import json

ruta = '/content/drive/MyDrive/PI MLOps - STEAM_nuevo/Datasets/output_steam_games.json'


with open(ruta, 'r') as f:
  steam_games_data = [json.loads(line) for line in f] # cada linea/fila será un diccionario json


In [34]:
# Crear el dataframe

df_steam_games = pd.DataFrame(steam_games_data) # cada fila corresponderá a los datos de un respectivo user_id

In [35]:
df_steam_games.head(3)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
df_steam_games.tail(3)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"
120444,NaN,NaN,Maze Run VR,NaN,http://store.steampowered.com/app/681550/Maze_...,NaN,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,True,681550,NaN


## Hay una gran cantidad de datos nulos en muchas columnas, tendremos que eliminar, varias columnas y filas

In [37]:
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


## Renombro la columna id para que matchee , con el resto de los dataframes

## Renombro la columna app_name como item_name para que machee con el resto de los datasets

In [38]:
df_steam_games.rename(columns={'id': 'item_id' , 'app_name' : 'item_name'}, inplace = True)

In [39]:
df_steam_games['price'] = pd.to_numeric(df_steam_games['price'], errors='coerce')


In [40]:
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   item_name     32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         28848 non-null  float64
 10  early_access  32135 non-null  object 
 11  item_id       32133 non-null  object 
 12  developer     28836 non-null  object 
dtypes: float64(1), object(12)
memory usage: 11.9+ MB


In [41]:
df_steam_games.tail()

,publisher,genres,item_name,title,url,release_date,tags,reviews_url,specs,price,early_access,item_id,developer
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"
120444,NaN,NaN,Maze Run VR,NaN,http://store.steampowered.com/app/681550/Maze_...,NaN,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,True,681550,NaN


## Guardo el dataframe steam para futuro analisis de EDA

In [42]:
df_steam_games.to_parquet('steam_raw.parquet')

## Elimino columnas que no utilizaré para el modelo y la API

In [43]:
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   item_name     32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         28848 non-null  float64
 10  early_access  32135 non-null  object 
 11  item_id       32133 non-null  object 
 12  developer     28836 non-null  object 
dtypes: float64(1), object(12)
memory usage: 11.9+ MB


In [44]:
df_steam_games.drop(['publisher', 'url', 'tags', 'reviews_url', 'specs'], axis=1, inplace = True)

In [45]:
df_steam_games.tail(10)

,genres,item_name,title,release_date,price,early_access,item_id,developer
120435,"[Action, Adventure, Indie]",The spy who shot me™,The spy who shot me™,2018-10-01,NaN,False,771810,Retro Army Limited
120436,"[Casual, Indie]",Raining blocks,Raining blocks,2018-01-04,0.99,False,767590,"Oscar Ortigueira López,OrtiGames/OrtiSoft"
120437,"[Indie, RPG]",Bravium,Bravium,2018-01-04,14.99,False,747320,INGAME
120438,"[Casual, Indie]",BAE 2,BAE 2,2018-01-04,NaN,False,769330,Riviysky
120439,"[Action, Adventure, Casual, Indie]",Kebab it Up!,Kebab it Up!,2018-01-04,1.99,False,745400,Bidoniera Games
120440,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,2018-01-04,1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,2018-01-04,4.99,False,733530,Sacada
120442,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,2018-01-04,1.99,False,610660,Laush Dmitriy Sergeevich
120443,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,2017-09-02,4.99,False,658870,"xropi,stev3ns"
120444,NaN,Maze Run VR,NaN,NaN,4.99,True,681550,NaN


In [46]:
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        28852 non-null  object 
 1   item_name     32133 non-null  object 
 2   title         30085 non-null  object 
 3   release_date  30068 non-null  object 
 4   price         28848 non-null  float64
 5   early_access  32135 non-null  object 
 6   item_id       32133 non-null  object 
 7   developer     28836 non-null  object 
dtypes: float64(1), object(7)
memory usage: 7.4+ MB


In [47]:
df_steam_games.drop(columns=['title', 'early_access'], inplace = True)

In [48]:
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        28852 non-null  object 
 1   item_name     32133 non-null  object 
 2   release_date  30068 non-null  object 
 3   price         28848 non-null  float64
 4   item_id       32133 non-null  object 
 5   developer     28836 non-null  object 
dtypes: float64(1), object(5)
memory usage: 5.5+ MB


## Elimino los valores nulos , de las columna mas relevantes para el modelo

In [49]:
df_steam_games.dropna(subset=['item_id', 'release_date', 'genres', 'item_name'], inplace = True)

In [50]:
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28832 entries, 88310 to 120443
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        28832 non-null  object 
 1   item_name     28832 non-null  object 
 2   release_date  28832 non-null  object 
 3   price         26076 non-null  float64
 4   item_id       28832 non-null  object 
 5   developer     28663 non-null  object 
dtypes: float64(1), object(5)
memory usage: 1.5+ MB


In [51]:
df_steam_games['price'].fillna(df_steam_games['price'].median(), inplace = True)
df_steam_games['developer'].fillna('', inplace = True)

In [52]:
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28832 entries, 88310 to 120443
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        28832 non-null  object 
 1   item_name     28832 non-null  object 
 2   release_date  28832 non-null  object 
 3   price         28832 non-null  float64
 4   item_id       28832 non-null  object 
 5   developer     28832 non-null  object 
dtypes: float64(1), object(5)
memory usage: 1.5+ MB


In [53]:
df_steam_games.head()

,genres,item_name,release_date,price,item_id,developer
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,4.99,643980,Secret Level SRL
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,4.99,670290,Poolians.com
88313,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,0.99,767400,彼岸领域
88315,"[Action, Adventure, Simulation]",Battle Royale Trainer,2018-01-04,3.99,772540,Trickjump Games Ltd


In [54]:
df_steam_games['year'] = df_steam_games['release_date'].str.split('-').str[0]


In [55]:
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28832 entries, 88310 to 120443
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        28832 non-null  object 
 1   item_name     28832 non-null  object 
 2   release_date  28832 non-null  object 
 3   price         28832 non-null  float64
 4   item_id       28832 non-null  object 
 5   developer     28832 non-null  object 
 6   year          28832 non-null  object 
dtypes: float64(1), object(6)
memory usage: 1.8+ MB


In [56]:
df_steam_games.head()

,genres,item_name,release_date,price,item_id,developer,year
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,4.99,643980,Secret Level SRL,2018
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,4.99,670290,Poolians.com,2017
88313,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,0.99,767400,彼岸领域,2017
88315,"[Action, Adventure, Simulation]",Battle Royale Trainer,2018-01-04,3.99,772540,Trickjump Games Ltd,2018


## Guardo el dataframe ya limpio como archivo parquet

In [57]:
df_steam_games.to_parquet('steam_games.parquet')

# Importamos el Archivo Items y lo desanidamos

In [58]:
ruta_items = '/content/drive/MyDrive/PI MLOps - STEAM_nuevo/Datasets/australian_users_items.json'

In [59]:
with open(ruta_items , 'r',encoding = 'utf-8') as f:
  data = f.readlines()


In [60]:
# data[0]

In [61]:
records= [eval(line.strip()) for line in data]

In [62]:
#records[0]

In [63]:
df_items_anidado = pd.DataFrame(records)


In [64]:
df_items_anidado.head(2)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."


In [65]:
#df_items_anidado['items'].iloc[0]

In [66]:
items_desanidado = []

for index, row in df_items_anidado.iterrows():
  user_id = row['user_id']
  items_count = row['items_count'] ,
  steam_id= row['steam_id'],
  user_url =  row['user_url'],
  items= row['items']

  for i in items  :
    new_row ={
    'user_id' : user_id,
    'items_count' : items_count[0] ,
    'steam_id' : steam_id[0] ,
    'user_url' :  user_url[0],
    'item_id': i.get('item_id')[0],
    'item_name': i.get('item_name', ''),
    'playtime_forever': i.get('playtime_forever'),
    'playtime_2weeks': i.get('playtime_2weeks')
    }
    items_desanidado.append(new_row)

df_items_desanidado = pd.DataFrame(items_desanidado)




In [67]:
df_items_desanidado['user_id'] = df_items_desanidado['user_id'].astype(str)

In [68]:
df_items_desanidado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 8 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   items_count       int64 
 2   steam_id          object
 3   user_url          object
 4   item_id           object
 5   item_name         object
 6   playtime_forever  int64 
 7   playtime_2weeks   int64 
dtypes: int64(3), object(5)
memory usage: 314.5+ MB


## No tiene datos nulos

In [69]:
nulos_por_columna = df_items_desanidado.isnull().sum()
print(nulos_por_columna)


user_id             0
items_count         0
steam_id            0
user_url            0
item_id             0
item_name           0
playtime_forever    0
playtime_2weeks     0
dtype: int64


In [70]:
df_items_desanidado.head()

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,Counter-Strike,6,0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2,Team Fortress Classic,0,0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,3,Day of Defeat,7,0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,4,Deathmatch Classic,0,0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,5,Half-Life: Opposing Force,0,0


In [71]:
df_items_desanidado.tail()

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
5153204,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,3,BrainBread 2,0,0
5153205,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,3,All Is Dust,0,0
5153206,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,3,One Way To Die: Steam Edition,3,3
5153207,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,5,You Have 10 Seconds 2,4,4
5153208,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,5,Minds Eyes,3,3


## Eliminamos las filas en las cuales los jugadores compraron el juego pero no lo jugaron

In [72]:
df_items = df_items_desanidado[df_items_desanidado['playtime_forever'] > 0].reset_index(drop=True)


In [73]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3285246 entries, 0 to 3285245
Data columns (total 8 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   items_count       int64 
 2   steam_id          object
 3   user_url          object
 4   item_id           object
 5   item_name         object
 6   playtime_forever  int64 
 7   playtime_2weeks   int64 
dtypes: int64(3), object(5)
memory usage: 200.5+ MB


In [74]:
df_items.head()

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,Counter-Strike,6,0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,3,Day of Defeat,7,0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,3,Day of Defeat: Source,4733,0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2,Counter-Strike: Source,1853,0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,3,Psychonauts,333,0


## Guardamos el archivo para el EDA

In [75]:
df_items.to_parquet('items_raw.parquet')

## Continuamos el ETL, eliminando columnas que no vamos a usar

In [76]:
df_items.drop(['steam_id','user_url', 'playtime_2weeks'], axis=1, inplace = True)

In [77]:
df_items.head(2)

,user_id,items_count,item_id,item_name,playtime_forever
0,76561197970982479,277,1,Counter-Strike,6
1,76561197970982479,277,3,Day of Defeat,7


## Nos aseguramos que la columna user_id sea string , eliminamos duplicados

In [78]:
#df_steam_games['user_id'] = df_steam_games['user_id'].astype('str')
df_items['user_id'] = df_items['user_id'].astype('str')
df_reviews['user_id'] = df_reviews['user_id'].astype('str')


# Eliminar duplicados basados en columnas específicas y mantener la primera lista de géneros
df_steam_games.drop_duplicates(subset=['item_name', 'release_date', 'item_id'], keep='first', inplace=True)

df_items.drop_duplicates(inplace = True)
df_reviews.drop_duplicates(inplace = True)


In [79]:

# Convertir la columna 'genres' en filas individuales
df_steam_exploded = df_steam_games.explode('genres')

In [80]:
df_steam_exploded.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71496 entries, 88310 to 120443
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        71496 non-null  object 
 1   item_name     71496 non-null  object 
 2   release_date  71496 non-null  object 
 3   price         71496 non-null  float64
 4   item_id       71496 non-null  object 
 5   developer     71496 non-null  object 
 6   year          71496 non-null  object 
dtypes: float64(1), object(6)
memory usage: 4.4+ MB


In [81]:
df_steam_exploded.head(9)

,genres,item_name,release_date,price,item_id,developer,year
88310,Action,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018
88310,Casual,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018
88310,Indie,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018
88310,Simulation,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018
88310,Strategy,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018
88311,Free to Play,Ironbound,2018-01-04,4.99,643980,Secret Level SRL,2018
88311,Indie,Ironbound,2018-01-04,4.99,643980,Secret Level SRL,2018
88311,RPG,Ironbound,2018-01-04,4.99,643980,Secret Level SRL,2018
88311,Strategy,Ironbound,2018-01-04,4.99,643980,Secret Level SRL,2018


In [82]:
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28831 entries, 88310 to 120443
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        28831 non-null  object 
 1   item_name     28831 non-null  object 
 2   release_date  28831 non-null  object 
 3   price         28831 non-null  float64
 4   item_id       28831 non-null  object 
 5   developer     28831 non-null  object 
 6   year          28831 non-null  object 
dtypes: float64(1), object(6)
memory usage: 1.8+ MB


In [83]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3246337 entries, 0 to 3285245
Data columns (total 5 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   items_count       int64 
 2   item_id           object
 3   item_name         object
 4   playtime_forever  int64 
dtypes: int64(2), object(3)
memory usage: 148.6+ MB


In [84]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48498 entries, 0 to 59276
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      48498 non-null  object
 1   posted       48498 non-null  object
 2   item_id      48498 non-null  object
 3   recommend    48498 non-null  bool  
 4   review       48498 non-null  object
 5   year_posted  48498 non-null  object
dtypes: bool(1), object(5)
memory usage: 2.3+ MB


## Guardos los archivos finales

In [85]:
df_steam_games.dropna(inplace = True)

In [86]:
df_steam_exploded.dropna(inplace = True)

In [87]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48498 entries, 0 to 59276
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      48498 non-null  object
 1   posted       48498 non-null  object
 2   item_id      48498 non-null  object
 3   recommend    48498 non-null  bool  
 4   review       48498 non-null  object
 5   year_posted  48498 non-null  object
dtypes: bool(1), object(5)
memory usage: 2.3+ MB


In [88]:
# Convertir la columna 'item_id' a tipo de dato entero, convirtiendo valores no válidos a NaN
df_items['item_id'] = pd.to_numeric(df_items['item_id'], errors='coerce').astype('Int64')

# Verificar el cambio de tipo de dato
print(df_items.dtypes)
print('-----------------')


# Convertir la columna 'item_id' a tipo de dato entero, convirtiendo valores no válidos a NaN
df_reviews['item_id'] = pd.to_numeric(df_reviews['item_id'], errors='coerce').astype('Int64')

# Convertir la columna 'item_id' a tipo de dato entero, convirtiendo valores no válidos a NaN
#df_reviews['year_posted'] = pd.to_numeric(df_reviews['year_posted'], errors='coerce').astype('Int64')

# Verificar el cambio de tipo de dato
print(df_reviews.dtypes)
print('-----------------')


# Convertir la columna 'item_id' a tipo de dato entero, convirtiendo valores no válidos a NaN
df_steam_games['item_id'] = pd.to_numeric(df_steam_games['item_id'], errors='coerce').astype('Int64')

# Convertir la columna 'item_id' a tipo de dato entero, convirtiendo valores no válidos a NaN
#df_steam_games['year'] = pd.to_numeric(df_steam_games['year'], errors='coerce').astype('Int64')

# Verificar el cambio de tipo de dato
print(df_steam_games.dtypes)
print('-----------------')


# Convertir la columna 'item_id' a tipo de dato entero, convirtiendo valores no válidos a NaN
df_steam_exploded['item_id'] = pd.to_numeric(df_steam_exploded['item_id'], errors='coerce').astype('Int64')

# Convertir la columna 'item_id' a tipo de dato entero, convirtiendo valores no válidos a NaN
#df_steam_exploded['year'] = pd.to_numeric(df_steam_exploded['year'], errors='coerce').astype('Int64')

# Verificar el cambio de tipo de dato
print(df_steam_exploded.dtypes)
print('-----------------')




user_id             object
items_count          int64
item_id              Int64
item_name           object
playtime_forever     int64
dtype: object
-----------------
user_id        object
posted         object
item_id         Int64
recommend        bool
review         object
year_posted    object
dtype: object
-----------------
genres           object
item_name        object
release_date     object
price           float64
item_id           Int64
developer        object
year             object
dtype: object
-----------------
genres           object
item_name        object
release_date     object
price           float64
item_id           Int64
developer        object
year             object
dtype: object
-----------------


In [93]:
df_items_reduc = df_items.head(1_000_000).drop(columns=['items_count'])

In [97]:
df_items = df_items.drop(columns=['items_count'])

In [98]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3246337 entries, 0 to 3285245
Data columns (total 4 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   item_id           Int64 
 2   item_name         object
 3   playtime_forever  int64 
dtypes: Int64(1), int64(1), object(2)
memory usage: 126.9+ MB


In [99]:
df_items_reduc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 0 to 1014874
Data columns (total 4 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   user_id           1000000 non-null  object
 1   item_id           1000000 non-null  Int64 
 2   item_name         1000000 non-null  object
 3   playtime_forever  1000000 non-null  int64 
dtypes: Int64(1), int64(1), object(2)
memory usage: 39.1+ MB


In [100]:
df_steam_games.to_parquet('steam_games.parquet')
df_steam_exploded.to_parquet('steam_exploded.parquet')
df_items.to_parquet('items.parquet')
df_reviews.to_parquet('reviews.parquet')
df_items_reduc.to_parquet('items_reduc.parquet')


In [101]:
df_reviews.to_csv('reviews.csv', sep = '|', index = False)
df_steam_games.to_csv('steam_games.csv',  index = False)
df_items.to_csv('items.csv',  index = False)
df_steam_exploded.to_csv('steam_exploded.csv',  index = False)
df_items_reduc.to_parquet('items_reduc.csv', index = False)